In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

In [47]:
train = pd.read_csv("train 2.csv")
test  = pd.read_csv("test_Vges7qu.csv")

In [48]:
print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()

Train shape: (550068, 12)
Test shape: (233599, 12)


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [49]:
train["Stay_In_Current_City_Years"] = train["Stay_In_Current_City_Years"].astype(str).str.replace("+", "", regex=False).astype(int)
test["Stay_In_Current_City_Years"]  = test["Stay_In_Current_City_Years"].astype(str).str.replace("+", "", regex=False).astype(int)

In [50]:
for col in ["Product_Category_1", "Product_Category_2", "Product_Category_3"]:
    if col in train.columns:
        train[col] = train[col].fillna(-1)
        test[col]  = test[col].fillna(-1)

In [51]:
train["Gender"] = train["Gender"].map({"F":0, "M":1})
test["Gender"]  = test["Gender"].map({"F":0, "M":1})

In [52]:
age_map = {"0-17":0, "18-25":1, "26-35":2, "36-45":3, "46-50":4, "51-55":5, "55+":6}
train["Age"] = train["Age"].map(age_map)
test["Age"]  = test["Age"].map(age_map)

In [53]:
city_map = {"A":0, "B":1, "C":2}
train["City_Category"] = train["City_Category"].map(city_map)
test["City_Category"]  = test["City_Category"].map(city_map)

In [54]:
for col in ["User_ID", "Product_ID"]:
    freq = train[col].value_counts().to_dict()
    train[f"{col}_freq"] = train[col].map(freq)
    test[f"{col}_freq"]  = test[col].map(freq).fillna(0)

In [55]:
features = [
    "Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years",
    "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3",
    "User_ID_freq", "Product_ID_freq"
]

X = train[features]
y = train["Purchase"]
X_test = test[features]

In [56]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=6, n_estimators=200, random_state=42)

In [58]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("Validation RMSE:", rmse)

Validation RMSE: 2671.402977526125


In [59]:
model.fit(X, y)
test_pred = model.predict(X_test)

In [60]:
submission = test[["User_ID", "Product_ID"]].copy()
submission["Purchase"] = test_pred
submission.to_csv("black_friday.csv", index=False)

submission.head()

,User_ID,Product_ID,Purchase
0,1000004,P00128942,15851.376665
1,1000009,P00113442,10456.612658
2,1000010,P00288442,6267.131660
3,1000010,P00145342,2961.311697
4,1000011,P00053842,2709.994201
